# Food Rec
Play around with recommending food that'll satisfy nutritional requirements not yet met in the current food selection.

Right now looks for food that'll minimize squared distance between food set's nutritional content and (randomly generated here) daily recommended amounts.

Note: "Food set" here is defined as the set of the user's selected foods AND the currently possibly recommended food (see below).

## Import Libraries

In [106]:
import pandas as pd
import random

## Look at Data

In [3]:
data = pd.read_csv("./Uncovering-the-Nutritional-Landscape-of-Food/nutrient_foodname_amount.tsv", sep="\t")
display(data)

,nutrient_id,amount,fdc_id,name,unit_name,NDB_number,Food_Name
0,1110,0.000,167640,"Vitamin D (D2 + D3), International Units",IU,35155,"Blueberries, wild, raw (Alaska Native)†"
1,1170,0.265,167640,Pantothenic acid,MG,35155,"Blueberries, wild, raw (Alaska Native)†"
2,1114,0.000,167640,Vitamin D (D2 + D3),UG,35155,"Blueberries, wild, raw (Alaska Native)†"
3,1003,1.220,167640,Protein,G,35155,"Blueberries, wild, raw (Alaska Native)†"
4,1007,0.230,167640,Ash,G,35155,"Blueberries, wild, raw (Alaska Native)†"
...,...,...,...,...,...,...,...
49798,1101,0.010,175266,"Manganese, Mn",MG,17092,"Veal, composite of trimmed retail cuts, separa..."
49799,1104,0.000,175266,"Vitamin A, IU",IU,17092,"Veal, composite of trimmed retail cuts, separa..."
49800,1103,5.500,175266,"Selenium, Se",UG,17092,"Veal, composite of trimmed retail cuts, separa..."
49801,1105,0.000,175266,Retinol,UG,17092,"Veal, composite of trimmed retail cuts, separa..."


## Get List of All Foods and Nutrients in Our Dataset

In [150]:
uniqueFoods = data["Food_Name"].unique()
uniqueNutrients = data["name"].unique()
#display(uniqueFoodsList, uniqueNutrients)

## Simulation Data


In [12]:
def get_info(food):
    '''Return a DataFrame containing nutrient data of only the food passed in'''
    return data[data["Food_Name"] == food]

In [98]:
def get_nutrient(food, nutrient):
    '''Return the specific nutrient amount for a particular food and nutrient of interest'''
    thisFood = get_info(food)
    nutInfo = thisFood[thisFood["name"] == nutrient]["amount"]
    
    if(len(nutInfo) == 0):
        return 0
    
    return float(nutInfo.values)



In [118]:
simSelectedFoods = ["Smelt, dried (Alaska Native)", "Blueberries, wild, raw (Alaska Native)†", "Palm hearts, raw†", "Fish, mackerel, king, raw†"]
nutrientsOfInterest = ["Iron, Fe", "Cholesterol", "Calcium, Ca"]

nutrientData = {nutrient:0 for nutrient in nutrientsOfInterest}

# Go through each food of interest and gather data
for food in simSelectedFoods:
    
    # Nutrient data
    nutrientData = {nut:amt+get_nutrient(food, nut) for (nut,amt) in nutrientData.items()}
    
display(nutrientData)

{'Iron, Fe': 9.24, 'Cholesterol': 302.0, 'Calcium, Ca': 1662.0}

## Rec System

In [148]:
recAmounts = {nut:random.random() * random.randint(100, 2000) for nut in nutrientsOfInterest}
display(recAmounts)

{'Iron, Fe': 301.9939962489131,
 'Cholesterol': 315.9561509657621,
 'Calcium, Ca': 81.00735388642025}

In [152]:
# Initial search for food to fill nutritional gaps
# Goes through every food one by one
# Can optimize later, kind of brute force right now

# Minimize distance between currently selected foods' nutritional contents and the recommended amounts
# Minimize square "residuals"

# Right now, just recommend 1 single food

# Sum of square distances between this food set's nutritional content and the recommended amounts
# "[T]his food set" referring to the set of selected foods IN ADDITION TO a possible food addition
squaredDistances = {}

for food in uniqueFoods:
    if food in simSelectedFoods:
        continue # Skip foods already selected
        
    # Calculate sum of square distances of this food set
    _distance = 0
    for nut in nutrientData.keys():
        _totalAmount = get_nutrient(food, nut) + nutrientData[nut] # Amount of current nutrient from food to possibly recommend + currently selected foods
        _distance += (_totalAmount - recAmounts[nut])**2 # The distance squared between total nutrient amount in this food set and recommended amount
        
    # Record squared distance
    squaredDistances[food] = _distance
    
recommendedFood = min(squaredDistances, key = squaredDistances.get)
print(f"Recommended food for you: {recommendedFood}, {squaredDistances[recommendedFood]}") # Number outputted is just for reference purposes, expect huge distance because of wide range of possible recommended amounts (randomly set here) and since Calcium is like over 1600

Recommended food for you: Bear, black, meat (Alaska Native)†, 2581273.6059887223


In [153]:
display(get_info(recommendedFood))

,nutrient_id,amount,fdc_id,name,unit_name,NDB_number,Food_Name
17182,1004,8.30,169793,Total lipid (fat),G,35007,"Bear, black, meat (Alaska Native)†"
17183,1005,0.00,169793,"Carbohydrate, by difference",G,35007,"Bear, black, meat (Alaska Native)†"
17184,1008,155.00,169793,Energy,KCAL,35007,"Bear, black, meat (Alaska Native)†"
17185,1051,71.20,169793,Water,G,35007,"Bear, black, meat (Alaska Native)†"
17186,1167,3.20,169793,Niacin,MG,35007,"Bear, black, meat (Alaska Native)†"
17187,1003,20.10,169793,Protein,G,35007,"Bear, black, meat (Alaska Native)†"
17188,1007,0.40,169793,Ash,G,35007,"Bear, black, meat (Alaska Native)†"
17189,1062,649.00,169793,Energy,kJ,35007,"Bear, black, meat (Alaska Native)†"
17190,1089,7.20,169793,"Iron, Fe",MG,35007,"Bear, black, meat (Alaska Native)†"
17191,1091,162.00,169793,"Phosphorus, P",MG,35007,"Bear, black, meat (Alaska Native)†"


In [142]:
# Kind of slow
# Faster "hack" that'll also make it so that the same food isn't recommended all the time
# is by just looking at a random subset of all the unique foods

# To be updated